In [6]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
import os
import copy

from dgl.geometry import farthest_point_sampler
import open3d as o3d

from utils import fps, depth2fgpcd, pcd2pix, fps_np, downsample_pcd

In [7]:
env = 'mustard_bottle'
folder_path = f'ptcl_data/{env}'

In [8]:
pcd_all_list = []
extrinsic_matrixs = []
global_scale = 24

for i in range(1, 5):
    # load data
    cam_view = 'view_{}'.format(i)
    dir_path = os.path.join(folder_path, cam_view)
    
    # obs = np.load(os.path.join(dir_path, 'obs.npy'))
    camera_intrinsic_params = np.load(os.path.join(dir_path, 'camera_intrinsic_params.npy')) # [fx, fy, cx, cy]
    camera_extrinsic_matrix = np.load(os.path.join(dir_path, 'camera_extrinsic_matrix.npy'))
    print('i:', i)
    print('camera_intrinsic_params\n', camera_intrinsic_params)
    print('camera_extrinsic_matrix\n', camera_extrinsic_matrix)
    extrinsic_matrixs.append(camera_extrinsic_matrix)
    
    pcd = o3d.io.read_point_cloud(os.path.join(dir_path, 'fgpcd.pcd'))
    pcd_array = np.asarray(pcd.points)
    print(pcd_array.shape, pcd_array.mean(axis=0))
    
    pcd_all_list.append(pcd)
    

i: 1
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[-4.3711388e-08  0.0000000e+00 -1.0000000e+00  1.0000001e+00]
 [-7.0710677e-01  7.0710677e-01  3.0908620e-08 -3.5355332e-01]
 [ 7.0710677e-01  7.0710677e-01 -3.0908620e-08 -2.6464465e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
(34491, 3) [-0.53838174  0.28166347  0.95199813]
i: 2
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[ 1.          0.          0.          0.5       ]
 [ 0.          0.70710677 -0.70710677  0.7071067 ]
 [ 0.          0.70710677  0.70710677 -3.7071066 ]
 [ 0.          0.          0.          1.        ]]
(32149, 3) [-0.64263643  0.29875469  1.07166872]
i: 3
camera_intrinsic_params
 [869.11683083 869.11683083 360.         360.        ]
camera_extrinsic_matrix
 [[ 1.1924881e-08  0.0000000e+00  1.0000000e+00 -9.9999994e-01]
 [ 7.0710677e-01  7.0710677e-01 -8.4321634e-09  3.5355

In [9]:
o3d.visualization.draw_geometries(pcd_all_list)

In [10]:
# save to pcd file
o3d.io.write_point_cloud(os.path.join(folder_path, 'merged.pcd'), pcd)

True